<a href="https://colab.research.google.com/github/VaishnavRathod/traffic-sign-recognition/blob/main/Traffic_Sign_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://bitbucket.org/jadslim/german-traffic-signs/src/master/

In [ ]:
import numpy as np
import pandas as pd
import cv2 as cv
import pickle
import matplotlib.pyplot as plt
import math

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential

In [ ]:
# Loading dataset

with open('master/train.p', 'rb') as fd:
  train_data = pickle.load(fd)

with open('master/test.p', 'rb') as fd:
  test_data = pickle.load(fd)

with open('master/valid.p', 'rb') as fd:
  val_data = pickle.load(fd)

print("Train data : ", train_data['features'].shape[0])
print("Test data  : ", test_data['features'].shape[0])
print("Valid data : ", val_data['features'].shape[0])

In [ ]:
def preprocessing(img):
  img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
  img = cv.equalizeHist(img)
  img = img/255. 
  
  return img

In [ ]:
# Dataset split into train test valid and preprocessing it

x_train = train_data['features']
y_train = train_data['labels']

x_test  = test_data['features']
y_test  = test_data['labels']

x_val   = val_data['features']
y_val   = val_data['labels']

x_train = np.array(list(map(preprocessing, x_train)))
x_test  = np.array(list(map(preprocessing, x_test)))
x_val   = np.array(list(map(preprocessing, x_val)))

x_train = np.reshape(x_train, (len(x_train), 32,32,1))
x_test  = np.reshape(x_test, (len(x_test)  , 32,32,1))
x_val   = np.reshape(x_val, (len(x_val)    , 32,32,1))


In [ ]:
x_train.shape

In [ ]:
# Image augmentation for more variety of Images

train_datagen = ImageDataGenerator(width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.1,
                                   shear_range=0.1,
                                   rotation_range=10)

In [ ]:
# Converting labels dataset into ONE HOT ENCODED format
y_train = to_categorical(y_train, 43)
y_test  = to_categorical(y_test, 43)
y_val   = to_categorical(y_val, 43)


In [ ]:
y_train

In [ ]:
def create_model():
  model = Sequential()

  model.add(Conv2D(60, (5,5), input_shape = (32,32,1), activation='relu'))
  model.add(Conv2D(60, (5,5), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))

  model.add(Conv2D(30, (3,3), activation='relu'))
  model.add(Conv2D(30, (3,3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))

  model.add(Flatten())

  model.add(Dense(500, activation='relu'))
  model.add(Dropout(0.5))

  model.add(Dense(43, activation='softmax'))

  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  return model

model = create_model()

In [ ]:
model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=64),
                    steps_per_epoch=540, 
                    epochs= 10, 
                    validation_data= (x_val,y_val),
                    shuffle=True)

In [ ]:
hist = model.history.history

In [ ]:
hist['accuracy']

In [ ]:
plt.plot(hist['accuracy'])
plt.plot(hist['val_accuracy'])

plt.legend(['Training', 'Validation'])
plt.grid(True)

plt.title('Accuracy per Epoch')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.show()

In [ ]:
plt.plot(hist['loss'])
plt.plot(hist['val_loss'])

plt.legend(['Training', 'Validation'])
plt.grid(True)

plt.title('Loss per Epoch')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.show()

In [ ]:
score = model.evaluate(x_test, y_test)

In [ ]:
print('Testing Accuracy is :', str(score[1]*100)[:5] + '%')

In [ ]:
model.save('10_Epochs.h5')